In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pyodbc
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
# Redshift 연결 정보
host='dev-data-etl-rs.ct0xxcn9xq5g.ap-northeast-2.redshift.amazonaws.com'
dbname='data_analysis_db'
user='stella'
password='iWntan282E5jJQc'
port='5439'

In [3]:
# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)

In [4]:
# 커서 생성
cursor = conn.cursor()
# SQL 쿼리 실행 (예시: frigga_sample 테이블에서 모든 데이터를 조회)
cursor.execute("""                          
                    SELECT 
                        *
                    FROM 
                        "data_analysis_db"."da_schema"."t_frigga_broken_target"
                    WHERE 1=1
                    ORDER BY device_serial ASC
                ;
               """)

# 결과 가져오기
rows = cursor.fetchall()

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# DataFrame으로 변환
df_broken = pd.DataFrame(rows, columns=columns)

# 커서와 연결 종료
cursor.close()
conn.close()

In [5]:
df_broken

,device_serial,start_date,end_date,duration_time,duration_day,min_temp,max_temp,mean_temp,std_temp,min_humi,max_humi,mean_humi,std_humi,min_acc,max_acc,mean_acc,std_acc,q1_temp,q3_temp,temp_start,temp_end,q1_humi,q3_humi,humi_start,humi_end,q1_acc,q3_acc,acc_start,acc_end,ta_min,ta_max,ta_mean,ta_std,ta_q1,ta_q3,ws1_min,ws1_max,ws1_mean,ws1_std,ws1_q1,ws1_q3,ws10_min,ws10_max,ws10_mean,ws10_std,ws10_q1,ws10_q3,hm_min,hm_max,hm_mean,hm_std,hm_q1,hm_q3,alert_temp_sum,alert_humi_sum,alert_acc_sum,broken_yn
0,VC6M6002,2024-12-02 14:47:18,2024-12-11 23:37:18,224.83,9.00,1.80,21.00,9.36,11.57,48.00,80.00,67.81,5.29,1.00,1.40,1.02,0.05,6.90,11.70,-0.30,18.90,64.00,72.00,52.00,84.00,1.00,1.00,1.00,1.00,0.70,16.10,6.11,3.25,3.80,8.00,0.00,9.10,3.32,1.51,2.30,4.20,0.00,8.10,3.24,1.34,2.30,3.90,15.50,83.40,45.84,12.33,39.80,53.60,12,10,200,0
1,VC6M6003,2024-12-14 09:40:13,2025-01-23 23:50:13,974.17,40.00,-0.10,19.80,6.83,14.00,37.00,66.00,56.53,7.61,1.00,1.52,1.02,0.06,3.90,9.00,-3.75,16.65,49.00,62.00,29.50,81.50,1.00,1.01,0.98,1.02,-5.40,12.10,5.38,3.79,3.90,8.08,0.00,7.30,2.94,1.32,2.10,3.70,0.10,7.50,2.99,1.26,2.10,3.70,-99.20,100.00,42.83,19.78,36.10,48.05,24,0,148,1
2,VC6M6007,2024-12-09 11:00:46,2025-01-23 09:20:46,1078.33,44.00,-10.80,9.60,0.56,10.74,41.00,90.00,57.38,6.91,0.90,1.93,0.99,0.05,-1.30,2.70,-7.30,8.70,53.00,61.00,41.00,73.00,0.92,1.00,0.80,1.12,-15.20,10.00,-1.07,4.60,-3.90,2.40,0.00,7.40,1.89,1.26,1.00,2.60,0.00,7.20,1.89,1.16,1.10,2.50,23.40,93.80,67.88,16.22,55.10,81.00,102,126,8,1
3,VC6M6008,2024-11-29 20:12:20,2024-11-30 08:42:20,12.50,0.00,1.10,6.10,3.18,1.45,66.00,75.00,69.08,2.29,1.00,1.21,1.01,0.05,2.40,4.03,-0.04,6.46,67.00,71.00,61.00,77.00,1.00,1.00,1.00,1.00,0.80,4.40,2.92,1.11,2.30,3.70,0.00,3.10,1.49,1.04,0.22,2.40,0.00,4.20,1.42,1.18,0.38,2.00,67.30,97.30,82.79,9.39,75.20,89.00,0,0,4,0
4,VC6M600A,2024-12-01 03:08:46,2024-12-04 14:48:46,83.67,3.00,2.50,10.10,8.27,2.84,56.00,81.00,59.46,5.16,1.00,1.20,1.05,0.06,7.63,9.30,5.11,11.81,57.00,59.00,54.00,62.00,1.00,1.10,0.85,1.25,-1.50,6.90,3.81,1.37,3.00,4.90,0.00,7.60,3.39,1.69,2.30,4.60,0.00,6.60,3.50,1.62,2.30,4.70,29.60,96.70,56.66,22.22,37.50,79.00,6,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,VC6M61C3,2025-01-22 11:48:21,2025-01-22 16:38:21,4.83,0.00,5.90,7.70,7.04,0.31,58.00,62.00,59.07,1.01,1.00,1.30,1.05,0.10,6.70,7.50,5.50,8.70,58.00,60.00,55.00,63.00,1.00,1.01,0.99,1.02,2.10,4.40,3.44,0.88,2.70,4.10,0.00,2.60,1.37,0.62,0.93,1.50,0.30,2.50,1.51,0.58,1.03,2.03,-99.20,99.90,66.90,46.10,71.70,84.80,0,0,7,0
152,VC6M61C4,2025-01-22 03:49:34,2025-01-23 23:59:34,44.17,1.00,0.80,6.50,2.78,1.58,53.00,65.00,56.77,1.69,1.10,1.32,1.10,0.02,1.83,3.58,-0.80,6.20,56.00,57.00,54.50,58.50,1.10,1.10,1.10,1.10,-3.20,5.60,0.39,2.84,-1.60,3.00,0.00,2.70,1.06,0.86,0.23,1.80,0.00,2.60,1.05,0.81,0.23,1.60,-99.20,94.70,67.96,30.10,55.30,90.40,2,19,3,0
153,VC6M61ED,2025-01-23 21:41:35,2025-01-23 21:51:35,0.17,0.00,1.40,1.70,1.55,0.04,46.00,47.00,46.50,0.71,1.22,1.30,1.26,0.06,1.47,1.63,1.25,1.85,46.25,46.75,45.50,47.50,1.24,1.28,1.18,1.34,-2.00,-2.00,-2.00,0.00,-2.00,-2.00,0.50,0.50,0.50,0.00,0.50,0.50,0.60,0.60,0.60,0.00,0.60,0.60,72.20,72.20,72.20,0.00,72.20,72.20,0,0,0,0
154,VC6ML002,2024-12-09 11:08:42,2024-12-21 02:08:42,279.00,11.00,2.30,24.90,21.04,17.57,15.00,57.00,24.69,7.28,1.00,1.89,1.09,0.06,20.80,23.00,17.50,26.30,20.00,27.00,9.50,37.50,1.10,1.10,1.10,1.10,-8.40,10.10,0.32,3.09,-1.70,2.45,0.00,6.30,1.65,0.96,0.90,2.30,0.00,4.60,1.64,0.86,1.00,2.20,30.70,97.60,67.45,15.75,53.80,80.05,91,91,244,1
